In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2, suppress=True)

%matplotlib inline

In [2]:
# lImg_fname = 'data/stars/20151016-221239-859-1.png'
# rImg_fname = 'data/stars/20151016-221239-859-2.png'
# lImg_fname = 'data/stars/20160218-201934-437-1.png'
# rImg_fname = 'data/stars/20160218-201934-437-2.png'
# lImg_fname = 'data/stars/20160219-021934-593-1.png'
# rImg_fname = 'data/stars/20160219-021934-593-2.png'
# lImg_fname = 'data/stars/20160323-031848-875-1.png'
# rImg_fname = 'data/stars/20160323-031848-875-2.png'
# lImg_fname = 'data/stars/20160329-043931-296-1.png'
# rImg_fname = 'data/stars/20160329-043931-296-2.png'
lImg_fname = 'data/stars/20160507-000119-796-1.png'
rImg_fname = 'data/stars/20160507-000119-796-2.png'

In [3]:
lImg = Image.open(lImg_fname).convert('RGB')
rImg = Image.open(rImg_fname).convert('RGB')
l_data = lImg.load() # pixmap of lImg
r_data = rImg.load() # pixmap of rImg

print l_data[0, 1]

(26, 28, 15)


In [4]:
# lImg

In [5]:
# rImg

In [6]:
assert (lImg.size == rImg.size)

w, h = width, height = lImg.size


print w, h

3112 2328


In [7]:
x_c, y_c = (width) / 2., (height) / 2. # center coordinates

print x_c, y_c

1556.0 1164.0


In [8]:
# fname_640x480 = 'data/coords_640x480.txt'
# fname_3072x2304 = 'data/coords_3072x2304.txt'
# fname_real_3072 = 'data/real_stars_3072/real_stars_3072_data_41_pairs.txt'
fname_real_3072 = 'data/real_stars_3072/20160507_000119_stars_coords.txt'

fname = fname_real_3072
coords = np.loadtxt(fname)

In [9]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print N, M

16 4


In [10]:
print 'Input coordinates from %r: \n' % fname, coords, '\n'

Input coordinates from 'data/real_stars_3072/20160507_000119_stars_coords.txt': 
[[  358.   928.   632.   980.]
 [  394.   671.   665.   734.]
 [  779.   664.  1033.   716.]
 [  844.   864.  1098.   911.]
 [ 1078.   950.  1327.   990.]
 [ 1257.  1031.  1504.  1070.]
 [ 1545.   964.  1791.   998.]
 [ 2154.   929.  2409.   955.]
 [ 2216.  1199.  2475.  1229.]
 [ 2572.  1151.  2848.  1177.]
 [ 2674.   731.  2951.   740.]
 [ 2823.   217.  3107.   198.]
 [  851.  1905.  1112.  1935.]
 [  994.  1947.  1252.  1979.]
 [ 1533.  2084.  1788.  2123.]
 [ 1545.  1725.  1797.  1761.]] 



In [11]:
# Solve the system of linear equations via pseudo inversion

# Below lX, lY, rX, rY mean lX[i], lY[i], rX[i], rY[i]:
# rX = a*lX + b*lY + e - eps1*z_x(lX, lY) + eps2*z_x(rX, rY)
# rY = c*lX + d*lY + f - eps1*z_y(lX, lY) + eps2*z_y(rX, rY),

# where approximately(!):
# z_x = (x-x0)*[ (x-x0)^2 +(y-y0)^2 ] = z_x(rX, rY) = z_x(Lx, Ly)
# z_y = (y-y0)*[ (x-x0)^2 +(y-y0)^2 ] = z_y(rX, rY) = z_y(Lx, Ly)

# xi = A*z + nu,      where:

# xi.T = rX[0], rY[0] ... rX[N-1], rY[N-1]
# z.T = a, b, c, d, e, f, eps1, eps2  -- 6 affine coeff-s + 2 distortion coeffs
# nu -- vector of inaccuracy (?) I don't use it right now

# A = [ # L = N-1   -- Last 
# lX[0] lY[0]  0     0      1  0  -z_x(lX[0], lY[0])    z_x(rX[0], rY[0]) ;
# 0     0      lX[0] lY[0]  0  1  -z_y(lX[0], lY[0])    z_y(rX[0], rY[0])
# ...                                                          ... ;
# ...                                                          ... ;
# ...                                                          ... ;
# lX[L] lY[L]  0     0      1  0  -z_x(lX[L], lY[L])    z_x(rX[L], rY[L]) ;
# 0     0      lX[L] lY[L]  0  1  -z_y(lX[L], lY[L])    z_y(rX[L], rY[L]) . ]


In [12]:
xi = np.zeros(2*N)
z = np.zeros(8)
arr = np.zeros((2*N, 8)) # matrix A

In [13]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i]
    xi[2*i + 1] = rY[i]

In [14]:
for i in xrange(N): # fill the A matrix
    dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
    dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

    zx1 = (lX[i] - x_c) * dist_l
    zx2 = (rX[i] - x_c) * dist_r
#     zx1, zx2 = 0, 0 # for simple affine6 algorithm!; comment out other tmps
    arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]
    
    zy1 = (lY[i] - y_c) * dist_l
    zy2 = (rY[i] - y_c) * dist_r
#     zy1, zy2 = 0, 0 # for simple affine6 algorithm!; comment out other tmps    
    arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]


In [15]:
print 'xi = A*z + nu'
# print 'xi:\n', xi
# print 'A:\n', arr, '\n'


xi = A*z + nu


In [16]:
pinv_arr = np.linalg.pinv(arr) # Calculate pseudo inverted A
z = np.dot(pinv_arr, xi)

In [17]:
np.set_printoptions(precision=10, suppress=True)
print 'Result:'
print 'z:', z
np.set_printoptions(precision=2, suppress=True)

Result:
z: [   0.9965012001    0.007705931    -0.0133277499    0.9992171143
  240.3708607634   54.535133319     0.0000000291    0.0000000293]


In [18]:
for i in xrange(width):
    for j in xrange(height):
        r, g, b = l_data[i, j]
        l_data[i, j] = r, 0, 0  # color lImg in red
        
        r, g, b = r_data[i, j]
        r_data[i, j] = 0, g, 0  # color rImg in green

In [19]:
l_data[x_c, y_c] = 255, 255, 255    # Mark the center
l_data[x_c-1, y_c] = 255, 255, 255
l_data[x_c+1, y_c] = 255, 255, 255
l_data[x_c, y_c-1] = 255, 255, 255
l_data[x_c, y_c+1] = 255, 255, 255

In [20]:
Image.blend(lImg, rImg, alpha=0.5) # Initial blend

In [81]:
a = float(z[0])
b = float(z[1])
c = float(z[2])
d = float(z[3])
e = float(z[4])
f = float(z[5])
eps1 = float(z[6])
eps2 = float(z[7])
data = (a, b, e, c, d, f) # affine coefficients

In [82]:
# aff_blend.show()

In [83]:
# nlImg = lImg.copy()
# nrImg = rImg.copy()
nlImg = Image.new('RGB', lImg.size, 'black')
nrImg = Image.new('RGB', rImg.size, 'black')
nl_data = nlImg.load()
nr_data = nrImg.load()

In [84]:
# Get rid of distortions on nlImg

for i in xrange(width): # xi
    for j in xrange(height): # eta
        dist_l = (i-x_c)**2 + (j-y_c)**2
                
        zx1 = (i - x_c) * dist_l
        zy1 = (j - y_c) * dist_l
        
        nl_data[round(i - eps1*zx1), round(j - eps2*zy1)] = l_data[i, j]


In [85]:
# Get rid of distortions on nrImg

for i in xrange(width): # xi
    for j in xrange(height): # eta
        dist_r = (i-x_c)**2 + (j-y_c)**2
                
        zx2 = (i - x_c) * dist_r
        zy2 = (j - y_c) * dist_r
        
        nr_data[round(i - eps1*zx2), round(j - eps2*zy2)] = r_data[i, j]


In [86]:
# nlImg.show()
# nrImg.show()

In [87]:
aff_nrImg = nrImg.transform(nrImg.size, Image.AFFINE, data, 
               resample=Image.BICUBIC)

aff_blend = Image.blend(nlImg, aff_nrImg, alpha=0.5) # only affine transform

aff_blend.save('aff_blend_temp.png')

In [88]:
# aff_blend